#### Pour que l'apprentissage par ML soit pertinent, on sélectionne les **catégories significatives** avec un "grand" nombre d'exposés des faits associés.

## La démarche proposée

- on crée des modèles Doc2Vec et on leur associe comme vocabulaire les exposés des faits distincts (sans doublons pour lesquels on connaît une ou plusieurs "bonnes" catégories)
- on entraîne ces modèles ainsi obtenus avec comme documents les **exposés des faits distincts ayant comme tags des identifiants uniques** => on aura autant de vecteurs numériques que d'exposés distincts
- avec chaque modèle (entrainé), on infère les vecteurs associés à de nouveaux exposés des faits à classifier (on réalise l'embedding ou la vectorisation des exposés)
- on demande à chaque modèle de trouver quels sont les exposés des faits (de son vocabulaire) les plus similaires avec chaque nouvel exposé des faits à classifier
- on propose comme catégories recommandées pour chaque nouvel exposé des faits les catégories des premiers exposés des faits déjà labélisés (du vocabulaire) qui sont les plus similaires avec le nouvel exposé

In [1]:
import random
import pickle
import numpy as np
import pandas as pd

from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from textblob import TextBlob as tb
from operator import itemgetter

import spacy
nlp = spacy.load('fr')
spacy.load('en')

# Embedding avec Doc2Vec

Dans l'architecture Doc2Vec, il y a 2 algorithmes : 
- **DBOW** (distributed bag of words) qui correspond à l'algorithme SG (skip-gram) de l'architecture Word2Vec
- **DM** (distributed memory) qui correspond à l'algorithme CBOW (continuous bag of words) de l'architecture Word2Vec

Avec **DBOW**, les vecteurs correspondant aux documents (paragraph vectors) sont obtenus en entraînant un réseau neuronal dont la tâche est de prédire une distribution de probabilité de mots dans un document (paragraphe) donné étant donné (à partir de) un échantillonage aléatoire de mots du paragraphe.

L'algorithme **DM** se comporte comme une mémoire qui se souvient de ce qui manque dans le contexte courant ou du sujet du document (paragraphe). Ainsi, les vecteurs correspondant aux documents (paragraphes) ont comme but de représenter le concept d'un document entier (et non pas des mots isolés).

## Chargement des données

**l_main_cats** est une liste avec les catégories principales, i.e. avec plus de 1000 exposés associés

In [2]:
with open('l_main_cats_51', 'rb') as f:
    l_main_cats = pickle.load(f)
print(l_main_cats[:3])
print(len(l_main_cats))

["DEMANDE - D'ASSISTANCE", 'TAPAGE NOCTURNE', 'ACCIDENT - DE CIRCULATION - DEGATS MATERIEL']
37


**main_efs** est un DataFrame contenant (seulement) les efs associés aux catégories "principales" (avec plus de 1000 edf correspondants)

In [3]:
main_efs = pd.read_csv('/home/alina/UNIFR/TM/random/ExposesDesFaits/main_efs_51.csv', header=0, sep=';')
main_efs.head(5)

,EXPOSES,CATEGORIE
0,Un renard sur la voie gauche.,ANIMAUX
1,Bras cassé suite glissade sur le verglas.,DEMANDE - D'AMBULANCE
2,Demande d'ambulance pour un jeune qui a trop b...,DEMANDE - D'AMBULANCE
3,"Un véhicule noire, plaques AI commençant évent...",ACCIDENT - DE CIRCULATION - AVEC FUITE
4,panne moteur,AR - PANNE - VEHICULE


## Exposés sans doublons

**ser_expos** correspond aux exposés de tous les enregistrements (donc les éléments de la colonne EXPOSES) de la DataFrame **efs** en tant qu'objet de type **pandas.core.series.Series**

In [4]:
ser_expos = main_efs['EXPOSES']
#ser_cats = efs.EXPOSES
print(type(ser_expos))

<class 'pandas.core.series.Series'>


**l_expos** est la liste de strings correspondant à la Series **ser_expos**

In [5]:
l_expos = list(ser_expos)
print(len(l_expos))
l_expos[0:3]

122204


['Un renard sur la voie gauche.',
 'Bras cassé suite glissade sur le verglas.',
 "Demande d'ambulance pour un jeune qui a trop bu et qui ne se réveille pas. Appel transféré au 144."]

On transforme la Series **ser_expos** en un set nommé **set_expos** pour éliminer les doublons

In [6]:
set_expos = set(ser_expos)
print(type(set_expos))
for i,expo in enumerate(set_expos):
    print(expo)
    if i>1:
        break

<class 'set'>
Retrouve son vhc endommagé sur le parking. N'a pas d'élément sur l'autre vhc impliqué. Dégâts de frottement uniquement.
Des vols auraient été commis dans les vestiaires, pendant le match de foot.
Une cliente fait du scandale et ne veut pas quitter le magasin.


On compte le nombre de doublons

In [7]:
print('Il reste', len(set_expos), 'exposés sans doublons.')
print('On a éliminé', len(ser_expos)-len(set_expos), 'doublons!')

Il reste 105930 exposés sans doublons.
On a éliminé 16274 doublons!


**l_set_expos** est la liste des exposés sans doublons

In [8]:
l_set_expos = list(set_expos)

On peut obtenir un nouveau DataFrame correspondant seulement aux enregistrements dont la colonne EXPOSES a une certaine valeur.

On considère par exemple l'exposé d'indice 12122 dans la liste des exposés sans doublons **l_set_expos**. 

In [9]:
n = 95783
l_set_expos[n]

'constate de nombreux dommages sur le refuge'

Les indices des enregistrements dans la DataFrame **main_efs** correspondant aux enregistrements dont l'exposé a une certaine valeur, dans cet exemple l'exposé d'indice ***n*** dans la liste des exposés sans doublons **l_set_expos**.

In [10]:
indices_expoN = main_efs.index[main_efs['EXPOSES']==l_set_expos[n]].tolist()
print(len(indices_expoN))
indices_expoN[0:10]

1


[91330]

Un nouveau DataFrame contenant seulement les enregistrements dont la colonne EXPOSES a une certaine valeur

In [11]:
(main_efs['EXPOSES']==l_set_expos[n])[:3]

0    False
1    False
2    False
Name: EXPOSES, dtype: bool

In [12]:
efs_expoN = main_efs.loc[main_efs['EXPOSES']==l_set_expos[n]]
print(len(efs_expoN))
efs_expoN.head()

1


,EXPOSES,CATEGORIE
91330,constate de nombreux dommages sur le refuge,DOMMAGES - A LA PROPRIETE


Le nombre d'apparitions d'un certain exposé

In [13]:
expoN_nbRows = len(efs_expoN)
print(expoN_nbRows)

1


**l_expoN_cats** est la liste des catégories associées à un certain exposé (avec doublons)

In [14]:
l_expoN_cats = list(efs_expoN['CATEGORIE'])
print(l_expoN_cats)

['DOMMAGES - A LA PROPRIETE']


**expoN_nbCats** est le nombre de catégories distinctes associées à un certain exposé

In [15]:
expoN_nbCats = len(set(l_expoN_cats))
print(expoN_nbCats)

1


## Création de la liste avec les TaggedDocument

**l_set_expos_rand** est obtenu par la randomization de la liste **l_set_expos**

In [16]:
l_set_expos_rand = random.sample(l_set_expos, len(l_set_expos))
print ("La liste originale : \n",  l_set_expos[:3])
print ("\nLa liste après la randomization non in place : \n",  l_set_expos_rand[:3])

La liste originale : 
 ["Retrouve son vhc endommagé sur le parking. N'a pas d'élément sur l'autre vhc impliqué. Dégâts de frottement uniquement.", 'Des vols auraient été commis dans les vestiaires, pendant le match de foot.', 'Une cliente fait du scandale et ne veut pas quitter le magasin.']

La liste après la randomization non in place : 
 ['A été mis en danger par la camionnette VD-73044. Veut déposer plainte. ', 'flammes dans gros container de chantier. Pompiers roulent', "A perdu son terrier herdal mâle de couleur marron et noir répondant au nom d'Oscar. A renseigné la SPA."]


**spacy_stop_words** est un set qui contient les "stop words" de la langue française

In [17]:
spacy_stop_words = spacy.lang.fr.stop_words.STOP_WORDS
print(type(spacy_stop_words))
print(len(spacy_stop_words))
#print(list(spacy_stop_words))

<class 'set'>
600


La fonction **clean** :
- reçoit en argument une liste d'exposés des faits
- retourne 2 listes :
    - une première liste avec les exposés des faits qui après tokenization et nettoyage gardent au moins 4 tokens
    - une deuxième liste de listes où la liste interne à la position i correspond aux tokens obtenus à partir de l'exposé des faits à la position i dans la première liste retournée
    
Pour le nettoyage notamment, on écrit tous les mots en minuscule, on élimine les stop words, les mots qui contiennent au moins un chiffre ainsi que les mots formés d'un seul caractère).

In [18]:
def clean(l_expos):
    ll_tokens = []
    l_expos_clean = []    
    for expo in l_expos:
        blob = tb(expo.lower())
        tokens = list(blob.words)
        # on élimine les stop words
        tokens = [elem for elem in tokens if elem not in spacy_stop_words]
        # on élimine les mots contenant au moins un chiffre
        tokens = [elem for elem in tokens if not any(c.isdigit() for c in elem)]
        # on élimine les mots formés d'un seul caractère
        tokens = [elem for elem in tokens if len(elem) > 1]
        # on garde les indices des edf avec seulement 1, 2 ou 3 tokens
        if(len(tokens)>=4):
            ll_tokens.append(tokens)
            l_expos_clean.append(expo)
    return l_expos_clean, ll_tokens

On tokenize et on "nettoie" les exposés des faits de la liste **l_set_expos_rand** et on obtient les listes **l_set_expos_rand_clean** et **ll_tokens**

In [19]:
%%time
l_set_expos_rand_clean, ll_tokens = clean(l_set_expos_rand)

CPU times: user 42.5 s, sys: 238 ms, total: 42.7 s
Wall time: 42.9 s


In [20]:
print('Le nombre d\'exposés restés après nettoyage :', len(l_set_expos_rand_clean))
print('La taille de la liste de listes de tokens :', len(ll_tokens)) 
print('Le nombre d\'edf avec 3 tokens ou moins :', len(l_set_expos_rand)- len(l_set_expos_rand_clean))

Le nombre d'exposés restés après nettoyage : 88694
La taille de la liste de listes de tokens : 88694
Le nombre d'edf avec 3 tokens ou moins : 17236


Vérification rapide pour contrôler la concordance des listes

In [21]:
li=[0, 45987, len(ll_tokens)-1] 
[print(l_set_expos_rand_clean[i]) for i in li]
[print(ll_tokens[i]) for i in li]

A été mis en danger par la camionnette VD-73044. Veut déposer plainte. 
Des sacs d'eau sont lancés contre les vitres de l'informatrice. 
Un client a commis des dommages et revient sur place. Serait menaçant.
['mis', 'danger', 'camionnette', 'veut', 'déposer', 'plainte']
['sacs', "d'eau", 'lancés', 'vitres', "l'informatrice"]
['client', 'commis', 'dommages', 'revient', 'place', 'menaçant']


[None, None, None]

La variable **l_tagged_docs** est une liste d'éléments de type **TaggedDocument**.

Plus précisément, chaque élément de cette liste est un objet **TaggedDocument** qui a comme attribut "**words**" une liste avec les mots d'un exposé des faits et comme attribut "**tags**" une liste avec un seul élément qui est l'indice du document transformé en String.

Le tag (qui est une liste avec un seul élément String) d'un document est en fait l'indice :
- de l'objet TaggedDocument correspondant dans la liste **l_tagged_docs**
- de la liste de tokens correspondante dans la liste **ll_tokens**
- du document (exposé) correspondant dans la liste **l_set_expos_rand_clean**

In [22]:
l_tagged_docs = [TaggedDocument(words=toks, tags=[str(i)]) for i, toks in enumerate(ll_tokens)]
print('Le nombre de tagged_documents :', len(l_tagged_docs))
#print(l_tagged_docs[0])
#print(l_tagged_docs[len(l_tagged_docs[0])-1])
[print(l_tagged_docs[i]) for i in li]

Le nombre de tagged_documents : 88694
TaggedDocument(['mis', 'danger', 'camionnette', 'veut', 'déposer', 'plainte'], ['0'])
TaggedDocument(['sacs', "d'eau", 'lancés', 'vitres', "l'informatrice"], ['45987'])
TaggedDocument(['client', 'commis', 'dommages', 'revient', 'place', 'menaçant'], ['88693'])


[None, None, None]

## Création d'un modèle abstrait Doc2Vec

On crée maintenant un modèle Doc2Vec abstrait qui a besoin de paramètres comme:
- **size** qui correspond au nombre de dimensions (features) des vecteurs numériques (feature vectors)
- **alpha** qui correspond au taux d'apprentissage (learning rate) initial
- **dm** qui précise l'algorithme d'entraînement - training algorithm (dm=1 pour "distributed memory" et dm=0 pour "distributed bag of words")
- **min_alpha** - durant l'entraînement, le taux d'apprentissage sera diminué linéairement jusqu'à cette valeur min_alpha
- **min_count** - on ignore tous les mots avec une "fréquence" totale plus petite que min_count 

Remarque: 
- l'algorithme Distributed Memory préserve l'ordre des mots dans le doument tandis que l'algorithme Distributed Bag Of Words emploie seulement l'approche standard Bag Of Words sans présérver l'ordre des mots
- Distributed Memory agit comme une mémoire qui se souvient de ce qui manque dans le contexte courant - ou du sujet du paragraphe
- Les "word vectors" correspondent à la notion de mot tandis que "document vectors" essaie de tenir compte du concept de document

### On crée un modèle Doc2Vec basé sur l'algorithme DM et une taille de 150 features par vecteur numérique

In [ ]:
vec_size = 150
alpha_in = 0.025
model_dm_150 = Doc2Vec(vector_size=vec_size, alpha=alpha_in, min_alpha=0.025,
                    min_count=1, dm=1, context = 10)

## Concrétisation du modèle abstrait 

On associe les données concrètes au modèle abstrait grâce à la méthode **build_vocab**

In [ ]:
#model_dm_150.build_vocab(l_tagged_docs)
model_dm_150.build_vocab([x for x in tqdm(l_tagged_docs)])
print(model_dm_150.corpus_count)
print(model_dm_150.epochs)

## Entraînement du modèle 

On entraine le modèle grâce surtout à la méthode **train**

In [ ]:
#model_dm_150.train(l_tagged_docs, total_examples=model_d2v.corpus_count, epochs=20) 

In [ ]:
%%time
max_epochs = 10
for epoch in range(max_epochs):
    #model_dm_150.train(l_tagged_docs, total_examples=model_d2v.corpus_count, epochs=model_d2v.epochs)
    model_dm_150.train([x for x in tqdm(l_tagged_docs)], total_examples=len(l_tagged_docs), epochs=model_dm_150.epochs)
    # on décroît le taux d'apprentissage
    model_dm_150.alpha -= 0.002
    model_dm_150.min_alpha = model_dm_150.alpha

## Deux autres modèles

### On crée et on entraîne un modèle Doc2Vec basé sur l'algorithme DBOW et une taille de 300 features par vecteur numérique

In [23]:
model_dbow_300 = Doc2Vec(dm=0, vector_size=300, alpha=0.065, min_alpha=0.065,
                   negative=5, hs=0, min_count=2, sample=0)

In [24]:
model_dbow_300.build_vocab([x for x in tqdm(l_tagged_docs)])

100%|██████████| 88694/88694 [00:00<00:00, 1945799.37it/s]


In [25]:
%%time
max_epochs = 30
for epoch in range(max_epochs):
    model_dbow_300.train([x for x in tqdm(l_tagged_docs)], total_examples=len(l_tagged_docs), epochs=1)
    # on décroît le taux d'apprentissage
    model_dbow_300.alpha -= 0.002
    model_dbow_300.min_alpha = model_dbow_300.alpha

100%|██████████| 88694/88694 [00:00<00:00, 2658236.74it/s]


CPU times: user 2min 21s, sys: 10.7 s, total: 2min 32s
Wall time: 1min 33s


In [26]:
print(len(model_dbow_300.docvecs))

88694


In [27]:
print(len(model_dbow_300.docvecs.vectors_docs))

88694


In [28]:
print(model_dbow_300.docvecs.vectors_docs.shape)

(88694, 300)


In [29]:
print(type(model_dbow_300.docvecs.doctags.items()))
list(model_dbow_300.docvecs.doctags.items())[:3]

<class 'dict_items'>


[('0', Doctag(offset=0, word_count=6, doc_count=1)),
 ('1', Doctag(offset=1, word_count=6, doc_count=1)),
 ('2', Doctag(offset=2, word_count=12, doc_count=1))]

On affiche un vecteur numérique en fonction de son indice 

In [30]:
print(model_dbow_300.docvecs[2][:5])
print(model_dbow_300.docvecs.vectors_docs[2][:5])

[-0.10954432 -0.20347948  0.10281222  0.21444637  0.11851917]
[-0.10954432 -0.20347948  0.10281222  0.21444637  0.11851917]


On affiche un vecteur numérique en fonction de son tag (qui, dans notre cas, est son indice pris comme String) 

In [31]:
print(model_dbow_300.docvecs['2'][:5])
#print(model_dbow_300.docvecs.vectors_docs['2'][:5])     #FAUX

[-0.10954432 -0.20347948  0.10281222  0.21444637  0.11851917]


La correspondance entre les indices et les tags

In [32]:
model_dbow_300.docvecs.index2entity[:5]

['0', '1', '2', '3', '4']

### On crée et on entraîne un modèle Doc2Vec basé sur l'algorithme DM et une taille de 300 features par vecteur numérique

In [ ]:
model_dm_300 = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, alpha=0.065, min_alpha=0.065,
                   negative=5, min_count=1)

In [ ]:
model_dm_300.build_vocab([x for x in tqdm(l_tagged_docs)])

In [ ]:
%%time
max_epochs = 30
for epoch in range(max_epochs):
    model_dm_300.train([x for x in tqdm(l_tagged_docs)], total_examples=len(l_tagged_docs), epochs=1)
    # on décroît le taux d'apprentissage
    model_dm_300.alpha -= 0.002
    model_dm_300.min_alpha = model_dm_300.alpha

Chaque modèle contient le même nombre de vecteurs numériques de "paragraphe" que le nombre de documents associés au modèle.

In [ ]:
print(type(model_dm_150.docvecs))
print(len(model_dm_150.docvecs))
print(len(model_dbow_300.docvecs))
print(len(model_dm_300.docvecs))

### Sauvegarde des modèles entraînés

In [ ]:
model_dm_150.save("model_dm_150_72.model")

In [33]:
model_dbow_300.save("model_dbow_300_72.model")

In [ ]:
model_dm_300.save("model_dm_300_72.model")

### Chargement du modèle sauvé

In [ ]:
model_dm_150= Doc2Vec.load("model_dm_150_71.model")

In [ ]:
model_dbow_300 = Doc2Vec.load("model_dbow_300_71.model")

In [ ]:
model_dm_300 = Doc2Vec.load("model_dm_300_71.model")

## Embedding d'un nouvel exposé des faits
Grâce au modèle déjà entraîné, on peut calculer le vecteur numérique (feature vector) d'un nouvel edf qui n'est pas dans l'ensemble d'entraînement

La fonction **edf2vec** reçoit comme paramètres:
- **expo** l'exposé des faits à vectoriser
- **model** le modèle Doc2Vec qui va inférer le vecteur numérique (feature vector)
- **epochs** le nombre d'itérations durant l'entraînement du nouveau document
- **seed** le "seed" pour la génération de nombres aléatoires pour le model utilisé

et elle retourne le nouveau feature vector dont la dimension est celle prévue par le modèle qui infère.

Remarque : des appels successifs de la méthode prédéfinie **infer_vector** peuvent retourner des vecteurs numériques différents et on recommande de choisir une valeur élevée pour le paramètre epochs afin de stabiliser la convergence.

In [34]:
def edf2vec(expo, model, epochs=None, seed=8017):
    if epochs is None:
        epochs=model.epochs
    blob = tb(expo.lower())
    doc_words = list(blob.words)
    spacy_stop_words = spacy.lang.fr.stop_words.STOP_WORDS
    # on élimine les stop words
    doc_words = [elem for elem in doc_words if elem not in spacy_stop_words]
    # on élimine les mots contenant au moins un chiffre
    doc_words = [elem for elem in doc_words if not any(c.isdigit() for c in elem)]
    # on élimine les mots formés d'un seul caractère
    doc_words = [elem for elem in doc_words if len(elem) > 1]
    #print(doc_words)
    model.random.seed(seed)
    v = model.infer_vector(doc_words, epochs)
    return v

In [35]:
edf_new = 'Une voiture est tombée en panne sur la route d\'Oron. Elle bloque la voie de droite de la route.'

In [ ]:
v_new_dm_150 = edf2vec(edf_new, model_dm_150)
print(edf_new, '\n')
print('Le vecteur inféré pour un document nouveau commence par :\n\n', v_new_dm_150[:4])

In [36]:
v_new_dbow_300 = edf2vec(edf_new, model_dbow_300)
print(edf_new, '\n')
print('Le vecteur inféré pour un document nouveau commence par :\n\n', v_new_dbow_300[:4])

Une voiture est tombée en panne sur la route d'Oron. Elle bloque la voie de droite de la route. 

Le vecteur inféré pour un document nouveau commence par :

 [-0.5834621   0.16971885  0.35350657  1.1821655 ]


In [ ]:
v_new_dm_300 = edf2vec(edf_new, model_dm_300)
print(edf_new, '\n')
print('Le vecteur inféré pour un document nouveau commence par :\n\n', v_new_dm_300[:4])

## Similarité des documents

La méthode **most_similar** permet de trouver les **topn** (qui par défaut vaut 10 et peut être donc omis) les plus similaires documents du modèle par rapport à un "document à comparer" qui est indiqué comme premier argument.

Le premier paramètre, donc le "document à comparer", peut être :
 - une string qui correspond au tag du document du modèle qui est comparé avec les autre documents du modèle
 - le nom **entre crochets** d'un feature vecteur qui ne fait pas partie du modèle mais qui a été inféré à partir du modèle

Le résultat retourné est une liste de couples dont la premier élément est le tag du document dans le modèle et le deuxième élément est sa similarité avec le "document à comparer" (et la liste est ordonnée de manière décroissante en fonction des similarités).

### La similarité entre les documents du modèle

On cherche dans chaque modèle les documents les plus similaires avec un certain document du modèle (pour lequel on donne son tag comme premier argument à la méthode **most_similar**)

D'abord, on considère le modèle **model_dbow_300** pour comprendre comment les vecteurs numériques sont stockés et comment on peut les retrouver.

In [ ]:
print(len(model_dbow_300.docvecs))
print(len(model_dbow_300.docvecs.vectors_docs))
print(model_dbow_300.docvecs.vectors_docs.shape)

L'attribut **model.docvecs.doctags** est un dictionnaire qui a comme clefs les tags et comme valeurs des objets Doctag.

In [37]:
#print(type(model_dbow_300.docvecs.doctags.items()))
list(model_dbow_300.docvecs.doctags.items())[:3]

[('0', Doctag(offset=0, word_count=6, doc_count=1)),
 ('1', Doctag(offset=1, word_count=6, doc_count=1)),
 ('2', Doctag(offset=2, word_count=12, doc_count=1))]

On peut retrouver un vecteur numérique en fonction de son indice (qui est une entier) dans **docvecs** ou dans **docvecs.vectors_docs**. 

In [38]:
#print(model_dbow_300.docvecs[2][:5])  #JUSTE
print(model_dbow_300.docvecs.vectors_docs[2][:5])

[-0.10954432 -0.20347948  0.10281222  0.21444637  0.11851917]


La méthode **index2entity** donne la correspondance entre les indices et les tags.

In [39]:
model_dbow_300.docvecs.index2entity[:5]

['0', '1', '2', '3', '4']

On peut retrouver un vecteur numérique en fonction de son tag (qui est une String) dans **docvecs** (mais pas dans docvecs.vectors_docs). 

In [40]:
print(model_dbow_300.docvecs['2'][:5])
#print(model_dbow_300.docvecs.vectors_docs['2'][:5])     #FAUX

[-0.10954432 -0.20347948  0.10281222  0.21444637  0.11851917]


La fonction **show_similar_expos_inner** permet de trouver les exposés (documents) du modèle les plus similaires avec un exposé (document) du modèle précisé par son tag et elle :
- reçoit comme arguments :
    - le modèle à utiliser
    - le tag (en tant que String) d'un exposé (document du vocabulaire du modèle) à comparer
    - le nombre d'exposés similaires à prendre en compte (par défaut 5)
- affiche les exposés similaires au document à comparer qui fait partie du modèle (avec leurs similarités)

In [41]:
def show_similar_expos_inner(model, tag, n=5):
    print('Exposé à comparer :\n"', l_set_expos_rand_clean[int(tag)], '"')
    #print(ll_tokens[int(tag)])
    similar_vects = model.docvecs.most_similar(tag, topn=n)
    #print('\nLes similaires :')
    #print(similar_docs)
    print('\nLes détails sur les documents similaires :')
    print('*******************************************')
    for tag_sym, similarity in similar_vects:
        indice = int(tag_sym)
        #print(ll_tokens[indice])
        print(l_set_expos_rand_clean[indice], '\nsimilarité :', similarity)
        print('-----------------------------------')

Exemples d'appels de la fonction **show_similar_expos_inner** pour différents modèles

In [42]:
#tag = '35248'
#tag = '65584'
tag = '77456'

In [43]:
show_similar_expos_inner(model_dbow_300, tag, n=5)

Exposé à comparer :
" Un renard mort sur le bord droit de la route "

Les détails sur les documents similaires :
*******************************************
Un renard mort au bord de la route. 
similarité : 0.9507027268409729
-----------------------------------
un renard mort sur le bord de la route 
similarité : 0.9501781463623047
-----------------------------------
Un chevreuil est mort sur la bord droit de la route. 
similarité : 0.9476631879806519
-----------------------------------
un renard mort au bord de la route 
similarité : 0.9463286399841309
-----------------------------------
1 renard mort sur bord de route 
similarité : 0.9406791925430298
-----------------------------------


/home/alina/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
show_similar_expos_inner(model_dm_150, tag, n=5)

In [ ]:
show_similar_expos_inner(model_dm_300, tag, n=5)

### Remarque

Il est important de pouvoir trouver les exposés des faits similaires entre eux car, par la suite, on va chercher pour un nouvel exposé des faits les exposés les plus similaires dans le vocabulaire du modèle - en partant du principe qu'**il y a une forte probabilité que deux exposés similaires aient justement la même catégorie**

### La similarité entre un nouvel exposé (nouveau document) et les exposés (documents) du modèle

On cherche dans le modèle les documents les plus similaires avec un nouveau document qui n'est pas dans le modèle (et pour lequel on donne son feature vector déjà calculé comme premier argument à la méthode **most_similar**)

Un exemple de résultat retourné par la méthode **most_similar** qui consiste en une liste de couples où le premier élément est le tag d'un exposé dans le modèle et le deuxième élément est la similarité entre cet exposé et le nouvel exposé (dont le vecteur numérique est passé comme premier argument de la méthode).

In [44]:
model_dbow_300.docvecs.most_similar([v_new_dbow_300], topn=5)

[('35478', 0.681040346622467),
 ('69011', 0.674630343914032),
 ('13978', 0.6632338166236877),
 ('86573', 0.6627055406570435),
 ('80124', 0.6582157611846924)]

La fonction **show_similar_docs** :
- reçoit comme arguments :
    - le modèle à utiliser
    - le vecteur numérique d'un nouveau edf à comparer
    - le nombre d'edf similaires à prendre en compte (par défaut 5)
- affiche les edf similaires au vecteur numérique à comparer (avec leurs similarités)

Ci-dessous, l'appel de la méthode **model.docvecs.most_similar** retourne une liste de couples où le premier élément d'un couple est le tag d'un vecteur similaire au vecteur argument et le deuxième élément est la similarité correspondante (et les couples sont présentés en ordre décroissant selon les similarités)

In [45]:
def show_similar_expos(model, vect, n=5):
    similar_vects = model.docvecs.most_similar([vect], topn=n)
    #print('\nLes similaires :')
    #print(similar_vects)
    print('\nLes détails sur les documents similaires :')
    print('*******************************************')
    for tag, similarity in similar_vects:
        indice = int(tag)
        print(ll_tokens[indice])
        print(l_set_expos_rand_clean[indice], '\nsimilarité :', similarity)
        print('-----------------------------------')

### Les catégories appropriées  pour un nouvel exposé (nouveau document)

La fonction **infer_categories** :
- reçoit comme arguments :
    - un exposé des faits
    - le DataFrame correspondant à la base de données (contenant des exposés avec leurs catégories associées) qui a servi à la création du modèle
- return la ou les catégories correspondantes à l'exposé des faits (avec les pourcents de leurs apparitions)

In [88]:
def infer_cats(expo, df):
    df_expo = df.loc[df['EXPOSES']==expo]
    l_cats = list(df_expo['CATEGORIE'])
    leng = len(l_cats)
    if leng == 1:
        #return l_cats[0]
        return [(l_cats[0], 1)]
    l_set_cats = list(set(l_cats))
    l_percents = []
    for cat_distinct in l_set_cats:
        i=0
        for cat in l_cats:
            if(cat==cat_distinct):
                i+=1
        l_percents.append(i/leng)
    l_couples = [(cat_distinct,l_percents[i]) for i, cat_distinct in enumerate(l_set_cats)]
    return sorted(l_couples, key=itemgetter(1), reverse=True)

Vérification de la méthode **infer_cats** pour des exposés qui ont été intégrés sans doublons dans le modèle Doc2Vec mais qui apparaissent dans la base de données avec 1, 2 ou plusieurs catégories associées

In [47]:
print(infer_cats('Un renard sur la voie gauche.', main_efs))

ANIMAUX


In [48]:
print(infer_cats('Bruit de comportement', main_efs))

[('TAPAGE NOCTURNE', 0.8), ('BRUIT', 0.2)]


In [49]:
print(infer_cats('Crevaison', main_efs))

[('AR - PANNE - VEHICULE', 0.9918032786885246), ('AR - ACCIDENT', 0.00819672131147541)]


In [50]:
print(infer_cats('2 voitures', main_efs))

[('ACCIDENT - DE CIRCULATION - DEGATS MATERIEL', 0.7904761904761904), ('AR - ACCIDENT', 0.2), ('ACCIDENT - DE CIRCULATION - AVEC BLESSE', 0.009523809523809525)]


La fonction **show_proposed_cats** :
- reçoit comme arguments :
    - le DataFrame correspondant à la base de données (contenant des exposés avec leurs catégories associées) qui a servi à la création du modèle
    - le modèle à utiliser
    - le vecteur numérique d'un nouveau edf à comparer
    - le nombre d'edf similaires à prendre en compte (par défaut 5)
- affiche les catégories similaires au vecteur numérique à comparer (avec leurs similarités)

In [51]:
def show_proposed_cats(df, model, vect, n=5):
    similar_vects = model.docvecs.most_similar([vect], topn=n)
    #print('\nLes similaires :')
    #print(similar_vets)
    print('\nLes détails sur les catégories similaires :\n')
    print('*******************************************')
    for tag, similarity in similar_vects:
        indice = int(tag)
        #print(ll_tokens[indice])
        expo = l_set_expos_rand_clean[indice]
        print(infer_cats(expo, df), '\nsimilarité :', similarity)
        print('----------------------------------')

### Tests pour des nouveaux exposés des faits

**edf_new** est un novel exposé des faits pour lequel on cherche des ***exposés similaires*** dans la base de données et les ***catégories appropriées***. 

In [52]:
#edf_new = 'Une voiture est en panne sur la route d\'Oron. Elle bloque la voie de droite de la route.'
edf_new = 'Les voisins font beaucoup trop de bruit. La musique est trop forte.'

Pour chaque modèle Doc2Vec, on infère (on calcule) le vecteur numérique correspondant au nouvel exposé des fait.s

In [ ]:
v_new_dm_150 = edf2vec(edf_new, model_dm_150)
#print(edf_new, '\n')
print('Le vecteur inféré commence par :', v_new_dm_150[:4])

In [53]:
v_new_dbow_300 = edf2vec(edf_new, model_dbow_300)
#print(edf_new, '\n')
print('Le vecteur inféré commence par :', v_new_dbow_300[:4])

Le vecteur inféré commence par : [-0.4050592  -0.18411472  0.32893413  1.1716193 ]


In [ ]:
v_new_dm_300 = edf2vec(edf_new, model_dm_300)
#print(edf_new, '\n')
print('Le vecteur inféré commence par :', v_new_dm_300[:4])

Pour un modèle donné et un nouvel exposé précisé par son vecteur numérique, on affiche des exposés similaires de la base de données, ainsi que leur degré de similarité.

In [54]:
show_similar_expos(model_dbow_300, v_new_dbow_300, 5)


Les détails sur les documents similaires :
*******************************************
['voisins', 'disputent', 'placards', 'claquent']
Les voisins se disputent. Les placards claquent. 
similarité : 0.7408130764961243
-----------------------------------
['voisins', 'bruit', 'waye', 'alkhaja']
ses voisins font du bruit (WAYE ALKHAJA) 
similarité : 0.736897349357605
-----------------------------------
['voisins', 'bougent', 'meubles', 'boucan']
Les voisins du dessus au 1er qui bougent les meubles et font du boucan depuis 3h. 
similarité : 0.7173482775688171
-----------------------------------
['voisins', 'fête', 'mi-balcon', 'mi-appart']
ses voisins du dessus font la fête mi-balcon, mi-appart. 
similarité : 0.7170052528381348
-----------------------------------
['voisins', 'bruit', 'sonner', 'bouchama']
Les voisins font trop de bruit au 2ème, sonner chez Bouchama. 
similarité : 0.7160227298736572
-----------------------------------


Pour un modèle donné et un nouvel exposé précisé par son vecteur numérique, on affiche des catégories associées dans la base de données aux exposés similaires, ainsi que leur degré de similarité.

In [55]:
show_proposed_cats(main_efs, model_dbow_300, v_new_dbow_300, 5)


Les détails sur les catégories similaires :

*******************************************
DEMANDE - D'ASSISTANCE 
similarité : 0.7408130764961243
----------------------------------
TAPAGE NOCTURNE 
similarité : 0.736897349357605
----------------------------------
BRUIT 
similarité : 0.7173482775688171
----------------------------------
TAPAGE NOCTURNE 
similarité : 0.7170052528381348
----------------------------------
TAPAGE NOCTURNE 
similarité : 0.7160227298736572
----------------------------------


## Sauvegarde de données

In [ ]:
model_dm_150.save("model_dm_150_72.model")

In [56]:
model_dbow_300.save("model_dbow_300_72.model")

In [ ]:
model_dbow_300.save("model_dbow_300_72.model")

In [58]:
with open('l_set_expos_rand_clean_72', 'wb') as f:
    pickle.dump(l_set_expos_rand_clean, f)

In [59]:
with open('ll_tokens_72', 'wb') as f:
    pickle.dump(ll_tokens, f)

In [60]:
with open('l_tagged_docs_72', 'wb') as f:
    pickle.dump(l_tagged_docs, f)

In [61]:
with open('l_main_cats_72', 'wb') as f:
    pickle.dump(l_main_cats, f)

In [57]:
main_efs.to_csv('/home/alina/UNIFR/TM/random/ExposesDesFaits/main_efs_72.csv', sep=';', index=False)

In [62]:
model_dbow_300_train = Doc2Vec(dm=0, vector_size=300, alpha=0.065, min_alpha=0.065,
                   negative=5, hs=0, min_count=2, sample=0)

In [63]:
l_tagged_docs_train, l_tagged_docs_test = \
            train_test_split(l_tagged_docs, test_size=0.1, random_state=2000)

In [64]:
model_dbow_300_train.build_vocab([x for x in tqdm(l_tagged_docs_train)])

100%|██████████| 79824/79824 [00:00<00:00, 2360998.56it/s]


In [65]:
%%time
max_epochs = 30
for epoch in range(max_epochs):
    model_dbow_300_train.train([x for x in tqdm(l_tagged_docs_train)], total_examples=len(l_tagged_docs_train), epochs=1)
    # on décroît le taux d'apprentissage
    model_dbow_300_train.alpha -= 0.002
    model_dbow_300_train.min_alpha = model_dbow_300_train.alpha

100%|██████████| 79824/79824 [00:00<00:00, 2476248.44it/s]


CPU times: user 2min 13s, sys: 9.97 s, total: 2min 23s
Wall time: 1min 29s


In [89]:
for tag_doc in l_tagged_docs_test:
    tag_test = tag_doc.tags[0]
    indice_test = int(tag_test)
    expo_test = l_set_expos_rand_clean[indice_test]
    #print(expo_test, '\n')
    l_cats_test = infer_cats(expo_test, main_efs)
    print(l_cats_test, '\n')

[('REVOCATION', 1)] 

[('REVOCATION', 1)] 

[('ACCIDENT - DE CIRCULATION - AVEC BLESSE', 1)] 

[('DROGUE', 1)] 

[('ACCIDENT - DE CIRCULATION - DEGATS MATERIEL', 1)] 

[('TAPAGE NOCTURNE', 1)] 

[('INDIVIDU - SUSPECT', 1)] 

[('ANIMAUX', 1)] 

[('MINEUR - IMPLIQUE', 1)] 

[("DEMANDE - D'AMBULANCE", 1)] 

[("VOL - A L'ETALAGE", 1)] 

[('VOL', 1)] 

[('ACCIDENT - DE CIRCULATION - AVEC BLESSE', 1)] 

[('ANIMAUX', 1)] 

[('INDESIRABLE', 1)] 

[('INDIVIDU - SUSPECT', 1)] 

[('MINEUR - IMPLIQUE', 0.5), ("DEMANDE - D'ASSISTANCE", 0.5)] 

[('ACCIDENT - DE CIRCULATION - DEGATS MATERIEL', 1)] 

[('VOL - PAR EFFRACTION', 1)] 

[('INCENDIE', 1)] 

[('VOL - PAR EFFRACTION', 0.5), ('TENTATIVE', 0.5)] 

[("DEMANDE - D'ASSISTANCE", 1)] 

[("DEMANDE - D'ASSISTANCE", 1)] 

[('INFRACTION - LCR', 1)] 

[('TAPAGE NOCTURNE', 1)] 

[('INFRACTION - LCR', 1)] 

[('INDESIRABLE', 1)] 

[('ACCIDENT - DE CIRCULATION - AVEC BLESSE', 1)] 

[('DROGUE', 1)] 

[('REVOCATION', 1)] 

[('AR - DIVERS', 1)] 

[('INDIVIDU - 

[('DROGUE', 0.5), ('TROUBLE - DE LA TRANQUILLITE / NUISANCE', 0.5)] 

[('BAGARRE', 1)] 

[('TROUBLE - DE LA TRANQUILLITE / NUISANCE', 1)] 

[('OPERATION', 1)] 

[("DEMANDE - D'ASSISTANCE", 1)] 

[('TAPAGE NOCTURNE', 1)] 

[("DEMANDE - D'ASSISTANCE", 1)] 

[("VOL - A L'ETALAGE", 1)] 

[('ACCIDENT - DE CIRCULATION - DEGATS MATERIEL', 1)] 

[('ACCIDENT - DE CIRCULATION - AVEC ANIMAL', 1)] 

[("VOL - A L'ETALAGE", 0.5), ('APPREHENSION / ARRESTATION', 0.5)] 

[('INDIVIDU - PERTURBE', 1)] 

[("DEMANDE - D'ASSISTANCE", 1)] 

[('INDIVIDU - PERTURBE', 0.5), ('REVOCATION', 0.5)] 

[('INFRACTION - LCR', 1)] 

[("DEMANDE - D'ASSISTANCE", 1)] 

[('TAPAGE NOCTURNE', 1)] 

[('ACCIDENT - DE CIRCULATION - DEGATS MATERIEL', 1)] 

[('COLLABORATION - INTERPOLICE', 0.5), ('OPERATION', 0.5)] 

[('MINEUR - IMPLIQUE', 0.5), ('REVOCATION', 0.5)] 

[("DEMANDE - D'ASSISTANCE", 1)] 

[('TROUBLE - DE LA TRANQUILLITE / NUISANCE', 1)] 

[('INDIVIDU - SUSPECT', 1)] 

[('COLLABORATION - INTERPOLICE', 1)] 

[("DEMANDE 

KeyboardInterrupt: 

In [112]:
def verify(l1, l2):
    l1_cats = [couple[0] for couple in l1]
    #print(l1_cats)
    l2_cats = [couple[0] for couple in l2]
    #print(l2_cats)
    l_common = list(set(l1_cats).intersection(l2_cats))
    #print(l_common)
    if(len(l_common) > 0):
        return True
    else:
        return False

In [113]:
l1 = [('MINEUR - IMPLIQUE', 0.3333333333333333), ('INDIVIDU - PERTURBE', 0.3333333333333333), ("DEMANDE - D'ASSISTANCE", 0.3333333333333333)] 

In [114]:
l2 = [('MINEUR - IMPLIQUE', 1)] 

In [115]:
verify(l1, l2)

True

In [128]:
hits = 0
for tag_doc in l_tagged_docs_test:
    tag_test = tag_doc.tags[0]
    indice_test = int(tag_test)
    expo_test = l_set_expos_rand_clean[indice_test]
    l_cats_test = infer_cats(expo_test, main_efs)
    print('--------------------\n', expo_test, '   ', l_cats_test)
    
    vect = model_dbow_300_train.infer_vector(tag_doc.words)
    
    similar_vects = model_dbow_300_train.docvecs.most_similar([vect], topn=3)
    
    for tag, similarity in similar_vects:
        indice = int(tag)
        expo = l_set_expos_rand_clean[indice]
        l_cats_sim = infer_cats(expo, main_efs)
        print(expo, '   ', l_cats_sim, '\n')
        if(verify(l_cats_test, l_cats_sim)):
            hits +=1
            break
        
print(hits)        

--------------------
 En fuite depuis le 20.08.2016 dès 2330, sous PLAFA, VD 1/3257, pourrait se trouver à son domcicile en compagnie de Mme ISAIAS DE SOUZA également signalée.      [('REVOCATION', 1)]
Placée sous PLAFA Signalée en fuite depuis le 17.08.2016 à 1955     [("FUITE - D'UN LIEU DE PLACEMENT", 0.5), ('REVOCATION', 0.5)] 

--------------------
 Avise que son garçon de 11 ans - Nathan - est resté dans le bus scolaire et n'est pas descendu à l'arrêt prévu. Est perturbé et a déjà eu des envies de suicide. A été hospitalisé dernièrement.  SIG : 11 ans, 110 cm, corpulence mince, cheveux , vêtu d'un T-shirt jaune et d'un short bleu, chaussé de baskets vertes.      [('REVOCATION', 1)]
1 H vêtu d'un masque de Spiderman     [('INDIVIDU - SUSPECT', 1)] 

Son fils Antoine (12 ans), 140 cm, corpulence mince, blonds, short bleu, t-shirt bleu a disparu depuis 1 heure.     [('REVOCATION', 1)] 

--------------------
 une voiture sur le flanc. Ambulance engagée par CTA. Pompiers envoient dési

un cycliste est tombé. Ambulance roule.     [('ACCIDENT - DE CIRCULATION - AVEC BLESSE', 1)] 

son mari est tombé et elle n'arrive pas à le relever      [("DEMANDE - D'ASSISTANCE", 1)] 

--------------------
 un vhc stationné hors case qui gêne la sortie de son vhc     [('INFRACTION - LCR', 1)]
vhc stationné sur domaine privé     [('INFRACTION - LCR', 1)] 

--------------------
 Un groupe de jeunes fait du bruit et écoute de la musique dans la cour du collège.     [('TAPAGE NOCTURNE', 1)]
Environ 6 jeunes qui font du bruit dans la cour du collège     [('TAPAGE NOCTURNE', 1)] 

--------------------
 des voitures au chantier sont mal parquées. L'informateur a été demander de bouger les véhicules. S'est fait "agresser" verbalement     [('INFRACTION - LCR', 1)]
plusieurs voitures empêchent l'informateur de sortir     [('CIRCULATION - ROUTIERE', 1)] 

plusieurs voitures mal parquées qui empêchent le passage     [('CIRCULATION - ROUTIERE', 1)] 

Un individu s'est fait frapper par un autre in

a recueilli un chat perdu qui semble perdu      [('ANIMAUX', 1)] 

--------------------
 signale un attroupement de personnes très bruyantes sur la rue. Serait là depuis 2 heures.     [('TAPAGE NOCTURNE', 1)]
signale un couple qui se disputent depuis 17 heures environ.      [('LITIGE', 1)] 

Des personnes qui crient en rue et ont des trompettes     [('BRUIT', 1)] 

2 personnes se battent. 4-5 personnes à proximité.     [('BAGARRE', 1)] 

--------------------
 Son bâtiment et le garage ont subi des dégâts     [('DOMMAGES - A LA PROPRIETE', 1)]
La porte de son garage individuel qui a été endommagée.     [('DOMMAGES - A LA PROPRIETE', 1)] 

--------------------
 Fourgon blanc plaques étrangères 80838-WX, comportement routier bizarre     [('CIRCULATION - ROUTIERE', 1)]
3 individus suspects avec comportement bizarre     [('INDIVIDU - SUSPECT', 1)] 

Comportement bizarre d'un individu.     [('INDIVIDU - SUSPECT', 1)] 

un homme au comportement suspect dans le quartier     [('INDIVIDU - SUSPE

Pour un homme qui dort dans le hall de l'immeuble.     [('INDESIRABLE', 1)] 

Un homme dort dans l'entrée de l'immeuble.     [('INDIVIDU - SUSPECT', 1)] 

Un individu dort devant l'entrée de l'immeuble.     [('INDESIRABLE', 1)] 

--------------------
 Auto - scooter. Douleurs à la jambe. Pas besoin d'ambulance.      [('ACCIDENT - DE CIRCULATION - AVEC BLESSE', 1)]
Auto - cycliste. Pas besoin d'ambulance.     [('ACCIDENT - DE CIRCULATION - AVEC BLESSE', 1)] 

--------------------
 Un individu (haut noir avec diamants dans le dos - pantalon gris - maillots de l'OM sous la veste noir - porte une casquette) porte une lame et est menaçant envers les passants. A même arrêté le trafic sur Bel-Air. Est passablement énervé.     [('INDESIRABLE', 1)]
Un individu ivre importune les passants     [('INDIVIDU - PERTURBE', 1)] 

Un individu habillé tout en noir insulte et frappe les gens.      [("DEMANDE - D'ASSISTANCE", 1)] 

un homme, habillé de foncé, qui parle avec les detenus     [("DEMANDE - D'A

--------------------
 Un homme visiblement ivre semble mal pris avec sa chaise roulante électrique.      [("DEMANDE - D'ASSISTANCE", 1)]
Un handicapé est tombé de sa chaise roulante, n'arrive pas à se relever     [("DEMANDE - D'ASSISTANCE", 1)] 

--------------------
 Un Renault Trafic blanc plaques BD751GK est stationné à l'entrée du chemin et cela parait suspect.     [('VEHICULE - SUSPECT', 1)]
véhicule suspect, véhicule français 4x4 blanc.      [('VEHICULE - SUSPECT', 1)] 

--------------------
 Retient une voiture qui sort de sa place de parc toute seule.     [('INFRACTION - LCR', 1)]
Retrouve sa voiture endommagée sur sa place de parc     [('ACCIDENT - DE CIRCULATION - AVEC FUITE', 1)] 

dans une voiture, sur place de parc 66. Informateur plus sur place, lésée pas sur place     [('VOL - PAR EFFRACTION', 0.5), ('VEHICULE', 0.5)] 

une voiture sans plaques est stationnée sur sa place de parc     [('VEHICULE', 1)] 

--------------------
 Une jeune fille qui se fait importuner par 5-6

...

--------------------
 A mis en fuite 2 jeunes. SIG : t-shirt blanc, pantalon court     [('VOL - PAR EFFRACTION', 0.3333333333333333), ('TENTATIVE', 0.3333333333333333), ('INDIVIDU - SUSPECT', 0.3333333333333333)]
musique à fond à l'arrêt de bus     [('TAPAGE NOCTURNE', 1)] 

1 H vêtu d'un masque de Spiderman     [('INDIVIDU - SUSPECT', 1)] 

--------------------
 Un homme se faisant passé pour un policier espagnole. Enquête pour des stups. L'informatrice trouve ça bizarre. Un homme mince, jean's et t-shirt bleu marine avec inscription blanche à longues manches.     [('INDIVIDU - SUSPECT', 1)]
Un homme avec un t-shirt bleu fait du porte-à-porte.     [('INDIVIDU - SUSPECT', 1)] 

--------------------
 Sa nièce s'est faite écrasée le pied par une dame qui a donné ses coordonnées.  Accident a eu lieu vers 1400. La fille a déjà été se faire soigner.     [('ACCIDENT - DE CIRCULATION - AVEC BLESSE', 1)]
MEMBRA 909 annonce un accident Chauderon-Tivoli. Auto-Scoot.     [('ACCIDENT - DE CIRCULAT

Chez Mme Iorilot qui casse tout dans l'appartement.      [("DEMANDE - D'ASSISTANCE", 1)] 

--------------------
 1 barre en métal sur voie gauche      [('AR - DIVERS', 1)]
un objet au terme de la voie d'engagement     [('AR - DIVERS', 1)] 

--------------------
 Un troupeau de vaches au milieu de la route.     [('ANIMAUX', 1)]
1 taurillon Highlands au milieu de la route     [('ANIMAUX', 1)] 

--------------------
 OP MCQUEEN, radar et LAPI, -> 2200     [('OPERATION', 1)]
OP VOLTA 2 - 3 véh. / 4 insp     [('OPERATION', 1)] 

--------------------
 Retrouve sa villa cambriolée pour la deuxième fois.      [('VOL - PAR EFFRACTION', 1)]
Retrouve sa maison cambriolée. Chez Niggler 079 625 52 39     [('VOL - PAR EFFRACTION', 1)] 

--------------------
 Une Mazda grise, sans plaque, est garée sur la place, remplie de matériel divers.     [('VEHICULE - SUSPECT', 1)]
une voiture sans plaque sur sa place de parc     [('CIRCULATION - ROUTIERE', 1)] 

Voiture sans plaque sur une place privée.     [(

Une MB noire (F) DT159KH fait des dépassements par la droite. Dangereux. Veut déposer plainte.     [('AR - DIVERS', 1)] 

--------------------
 Appréhension de deux individus suspects de vol. Demande une patrouille, car une mesure d'interdiction de centre va être prise contre eux.      [('INDIVIDU - SUSPECT', 1)]
demande K9 pour cambr mis en fuite     [('VOL - PAR EFFRACTION', 1)] 

demande une patrouille pour identifier un resquilleur     [('DEMANDE - IDENTIFICATION', 1)] 

CET demande une patrouille limite de secteur     [('COLLABORATION - INTERPOLICE', 1)] 

--------------------
 feu confirmé dans la fosse à poubelle suite détection automatique de l'Entreprise Valorsa     [('INCENDIE', 1)]
Pompiers roulent pour un feu de poubelle     [('INCENDIE', 1)] 

--------------------
 interpellation dans le cadre de l'OP Chaume     [('OPERATION', 1)]
MAM dans le cadre d'une enquête     [('APPREHENSION / ARRESTATION', 1)] 

Perquis et MAM dans le cadre de l'OP "DARYL"     [('OPERATION', 0.5), 

Blouson bleu marine, pull gris, pantalon foncé. Le 07.12.2016 à 1430, sous PLAFA.     [('REVOCATION', 1)] 

Un piéton habillé en foncé sur la BAU     [('AR - DIVERS', 1)] 

--------------------
 une personne âgée (1937) a chuté en descendant l'escalator. Consciente     [("DEMANDE - D'ASSISTANCE", 1)]
pour relever une dame âgée ayant chuté. Pas blessée     [("DEMANDE - D'ASSISTANCE", 1)] 

--------------------
 Mme PEREU ALIBRANDO Tatiana A des problèmes relationnels avec son mari, mais apparemment il n'y a pas eu de coup.     [("DEMANDE - D'ASSISTANCE", 1)]
a des problèmes avec son mari. Situation tendue     [("DEMANDE - D'ASSISTANCE", 1)] 

--------------------
 un homme OH++ qui s'endort sur la table et refuse de quitter les lieux     [('INDIVIDU - PERTURBE', 1)]
1 homme refuse de quitter les lieux.     [('INDESIRABLE', 1)] 

un homme OH++ qui refuse de quitter les lieux     [('INDESIRABLE', 1)] 

un homme refuse de quitter les lieux.     [('INDESIRABLE', 1)] 

--------------------
 

Cygne blessé, se trouve vers la sculpture.     [('ANIMAUX', 1)] 

--------------------
 entend des cris provenant du corridor     [("DEMANDE - D'ASSISTANCE", 1)]
Entend des cris qui proviennent de chez ses voisins.     [("DEMANDE - D'ASSISTANCE", 1)] 

--------------------
 Signale que le père de son fils est parti avec l'enfant au Portugal sans son accord. Garde partagée au Portugal mais statu non reconnu en Suisse     [('MINEUR - IMPLIQUE', 1)]
sa part en "vrille" entre son père et son frère     [("DEMANDE - D'ASSISTANCE", 1)] 

Vont chercher 2 enfants qui sont sous la garde de la SPJ     [('OPERATION', 1)] 

a vu des parents donner de la biere un enfant de 1an     [('MINEUR - IMPLIQUE', 1)] 

--------------------
 Un homme, pour vol dans le rayon parfumerie, pour un montant de CHF 303.80. Cet individu est déjà sous interdiction de magasin et parle le français.     [("VOL - A L'ETALAGE", 1)]
CHF 349.- majeur, plainte magasin, parle français.     [("VOL - A L'ETALAGE", 1)] 

---------

In [129]:
print(len(l_tagged_docs_test))
accuracy = hits/len(l_tagged_docs_test)*100
print(accuracy)

8870
59.17700112739571


## Extension du travail

- on crée des modèles Doc2Vec comme ci-dessus et on leur associe comme vocabulaire les exposés des faits déjà labelisés
- on entraine ces modèles ainsi obtenus avec comme documents les exposés des faits déjà labélisés
- on récupère les vecteurs numériques correspondant aux exposés des faits déjà labélisés
- avec le modèle Doc2Vec (entrainé), on infère le vecteur associé à un nouvel exposé de faits à classifier (on réalise l'embedding)
- avec tous ces vecteurs vus comme des vecteurs de features, on fait une classification supervisée grâce à un modèle LogisticRegression